In [1]:
import torchvision

In [4]:
print(torchvision.__version__)

0.20.1+cpu


In [3]:
class A:
    def __init__(self):
        print("A initialized")

class B(A):
    def __init__(self):
        super().__init__()
        print("B initialized")

class C(A):
    def __init__(self):
        super().__init__()
        print("C initialized")

class D(B, C):
    def __init__(self):
        super().__init__() # 必须手动调用父类的初始化函数
        print("D initialized")

# 创建 D 类的实例
d = D()

A initialized
C initialized
B initialized
D initialized


# 小实验

展示 train_loss += loss 和 train_loss += loss.item() 这两种写法最后 train_loss 的类型、显存使用的区别

In [1]:
import torch
import gc

# 一个简单的线性模型和数据
model = torch.nn.Linear(10, 1)
x = torch.randn(32, 10)
y = torch.randn(32, 1)
loss_fn = torch.nn.MSELoss()

# 情况1: 使用 loss 直接累加
train_loss_tensor = 0  # Python int
for _ in range(5):
    out = model(x)
    loss = loss_fn(out, y)
    train_loss_tensor += loss  # 会变成 Tensor
type_tensor_loss = type(train_loss_tensor)
is_tensor_requires_grad = getattr(train_loss_tensor, "requires_grad", None)

# 清理显存
del train_loss_tensor, out, loss
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# 情况2: 使用 loss.item() 累加
train_loss_float = 0.0  # Python float
for _ in range(5):
    out = model(x)
    loss = loss_fn(out, y)
    train_loss_float += loss.item()  # 纯 float
type_float_loss = type(train_loss_float)

(type_tensor_loss, is_tensor_requires_grad, type_float_loss)


(torch.Tensor, True, float)

📌 **总结**：

* `loss`：Tensor，带梯度，累加会“拖着计算图跑”。
* `loss.item()`：普通数值，累加安全、轻量，训练日志和统计指标都应该用这个。

# GPU


In [1]:
import torch

In [2]:
print(torch.cuda.is_available())

True


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


# random_split

In [ ]:
from torch.utils.data import random_split


# max vs. argmax

In [5]:
import torch
outputs = torch.tensor([[2.5, 1.0, 0.2],
                        [0.1, 3.2, 2.7]])

# 方法1: max
values, indices = outputs.max(1)
print(values)   # tensor([2.5, 3.2])
print(indices)  # tensor([0, 1])  -> 类别索引

# 方法2: argmax
predicted = outputs.argmax(1)
print(predicted)  # tensor([0, 1])


tensor([2.5000, 3.2000])
tensor([0, 1])
tensor([0, 1])


In [6]:
import torch

# 假设有一批预测结果和真实标签
predicted = torch.tensor([0, 2, 1, 3])
labels = torch.tensor([0, 1, 1, 3])

# 方法1: 使用 ==
correct_eq = (predicted == labels).sum().item()

# 方法2: 使用 .eq()
correct_method = predicted.eq(labels).sum().item()

# 输出结果和类型
(correct_eq, type(correct_eq), correct_method, type(correct_method))


(3, int, 3, int)

In [7]:
(predicted == labels)

tensor([ True, False,  True,  True])

In [8]:
predicted.eq(labels)

tensor([ True, False,  True,  True])

# tqdm

# 多个字符串字面量直接相邻放在一起

Python 允许多个字符串字面量直接相邻放在一起，Python 会自动把它们拼接成一个字符串。这个特性叫做 implicit string concatenation（隐式字符串拼接）。

In [ ]:
# --- Log ---
print(f"Epoch [{epoch+1}/{num_epochs}] "f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

# 选择最合适的num_workers值

In [3]:
from time import time
import multiprocessing as mp
import torch
import torchvision
from torchvision import transforms
from dataset import ClassDirectoryDataset
from torch.utils.data import DataLoader,random_split
 
transform = transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

dataset = ClassDirectoryDataset("data/hymenoptera_data/train", ["jpg"])
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size],
                                  generator=torch.Generator().manual_seed(42)
)


 
print(f"num of CPU: {mp.cpu_count()}")
for num_workers in range(2, mp.cpu_count(), 2):  
    train_loader = DataLoader(train_set,
                          batch_size=16,
                          num_workers=num_workers,
                          shuffle=True,
                          pin_memory=True  # Enable this option to accelerate data transfer if using GPU
)
    start = time()
    for epoch in range(1, 3):
        for i, data in enumerate(train_loader, 0):
            pass
    end = time()
    print("Finish with:{} second, num_workers={}".format(end - start, num_workers))

ModuleNotFoundError: No module named 'dataset'

# argparse

In [3]:
import argparse

NameError: name 'python' is not defined

In [1]:

parser = argparse.ArgumentParser(description="argparse hello world")
parser.add_argument("--lr",type=float,required=True,help="learning rate")
parser.add_argument("--batch_size",type=int,required=True,help="batch size")
parser.add_argument("--epochs",type=int,required=True,help="number of epochs")

args = parser.parse_args()
print(args)


usage: ipykernel_launcher.py [-h] --lr LR --batch_size BATCH_SIZE --epochs
                             EPOCHS
ipykernel_launcher.py: error: the following arguments are required: --lr, --batch_size, --epochs


SystemExit: 2

C:\Users\Cavendishes\.conda\envs\learn-pytorch\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# configs

In [ ]:
import yaml
yaml.safe_load(open("config.yaml","r"))